In [1]:
'''
Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet

complete list of built-in transforms: 
https://pytorch.org/docs/stable/torchvision/transforms.html

On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale

On Tensors
----------
LinearTransformation, Normalize, RandomErasing

Conversion
----------
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage

Generic
-------
Use Lambda 

Custom
------
Write own class

Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])
'''

'\nTransforms can be applied to PIL images, tensors, ndarrays, or custom data\nduring creation of the DataSet\n\ncomplete list of built-in transforms: \nhttps://pytorch.org/docs/stable/torchvision/transforms.html\n\nOn Images\n---------\nCenterCrop, Grayscale, Pad, RandomAffine\nRandomCrop, RandomHorizontalFlip, RandomRotation\nResize, Scale\n\nOn Tensors\n----------\nLinearTransformation, Normalize, RandomErasing\n\nConversion\n----------\nToPILImage: from tensor or ndrarray\nToTensor : from numpy.ndarray or PILImage\n\nGeneric\n-------\nUse Lambda \n\nCustom\n------\nWrite own class\n\nCompose multiple Transforms\n---------------------------\ncomposed = transforms.Compose([Rescale(256),\n                               RandomCrop(224)])\n'

In [2]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        # data loading
        xy = np.loadtxt('./data/wine/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        
        # note that we do not convert to tensor here
        self.x = xy[:, 1:] # 所有 samples 的第二列到最后一列
        self.y = xy[:, [0]] # 所有 samples 的第一列
        
        self.transform = transform
        
    def __getitem__(self, index):
        # dataset[0]
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def __len__(self):
        # len(dataset)
        return self.n_samples

In [4]:
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [11]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        inputs, target = sample
        inputs *= self.factor
        return inputs, target

In [15]:
dataset = WineDataset(transform=ToTensor())

first_data = dataset[0]
features, labels = first_data
print(features)
print(type(features), type(labels))

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])
<class 'torch.Tensor'> <class 'torch.Tensor'>


In [16]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform=composed)

first_data = dataset[0]
features, labels = first_data
print(features)
print(type(features), type(labels))

tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03])
<class 'torch.Tensor'> <class 'torch.Tensor'>
